In [ ]:
import glob
import codecs
from utils import in_blocklist

blocklist = []
blocklist.append('AWAYLEN=307 MAXTARGETS=20 WALLCHOPS WATCH=128 WATCHOPTS=A SILENCE=15 MODES=12 CHANTYPES=# PREFIX')
blocklist.append('- * -')
blocklist.append('XxXChatters.Com')
blocklist.append('XxXChatters')
blocklist.append('This server was created')
blocklist.append('operator(s) online')
blocklist.append('is now your displayed host')
blocklist.append('Caps set:')
blocklist.append('MAXCHANNELS')
blocklist.append('http')
blocklist.append('type it in the main room or a pm to the')
blocklist.append('The topic is')
blocklist.append('Topic set by')

all_all_posts = []

all_posts = []
all_posts_410 = []
all_posts_501 = []
authors = {}
authors_410 = {}
authors_501 = {}

for file in glob.glob("xxxchatters_logs/#chat*.log"):
#for file in glob.glob("xxxchatters_logs/*.log"):
    print(file)
    lines = codecs.open(file, mode='r', encoding='utf-8')
    
    for line in lines:
        if not in_blocklist(blocklist, line.strip()): # blocked?
            if line[0] is '[' and line.find("<") is not -1:
                s = line
                sentence = s[s.find(">")+2:]

                author = s[s.find("<")+1:s.find(">")]

                all_posts.append((author, sentence.strip().lower()))

                if author not in authors: # save things
                    authors[author] = set() # init list for author
                authors[author].add(sentence.strip().lower()) 
            else:
                print(line)

for line in codecs.open('1006/410_utf8.txt', mode='r', encoding='utf-8'):
    if ':' in line and not in_blocklist(blocklist, line.strip()): # blocked?:
        sentence = line[line.find(":"):]
        all_posts.append((author, sentence.strip().lower()))
        all_posts_410.append((author, sentence.strip().lower()))
        
        author = line[:line.find(":")]
        if author not in authors: # save things
            authors[author] = set() # init list for author
        if author not in authors_410:
            authors_410[author] = set()
        authors[author].add(sentence.strip().lower())
        authors_410[author].add(sentence.strip().lower()) 


for s in codecs.open("1006/501_utf8.txt", mode='r', encoding='utf-8').readlines():
    if ':' in s and 'has left the channel' not in s and 'has quit' not in s and 'joined the channel' not in s:
        if not in_blocklist(blocklist, s.strip()): # blocked?
            sentence = s[s.find(">")+2:]
            all_posts.append((author, sentence.strip().lower()))
            all_posts_501.append((author, sentence.strip().lower()))

            author = s[s.find("<")+1:s.find(">")]
            if author not in authors: # save things
                authors[author] = set() # init list for author
            if author not in authors_501:
                authors_501[author] = set()
            authors[author].add(sentence.strip().lower())
            authors_501[author].add(sentence.strip().lower()) 
        
print(len(all_posts))
print(len(authors))
print(len(authors_410))
print(len(authors_501))
print(len(all_posts_410))
print(len(all_posts_501))

In [ ]:
for a in authors['carrol']:
    print(a)

In [135]:
import operator
import string
word_counts = dict()
translator = str.maketrans('', '', string.punctuation)

def preprocess_word(word):
    word = word.replace('\x02', '')
    word = word.replace('\x03', '')
    word = word.replace('.', '')

    #word = word.translate(translator)
    word = ''.join(e for e in word if e.isalnum())
    return word


for post in all_posts:
    word_list = post[1].split()
    for word in word_list:
        word = preprocess_word(word)
        #word = word.replace('\x03', '')
        #word = word.replace('.', '')

        #word = word.translate(translator)
        #word = ''.join(e for e in word if e.isalnum())
        if len(word) < 1:
            continue
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1

sorted_word_counts = sorted(word_counts.items(), key=operator.itemgetter(1), reverse=True)

def get_sorted_word_counts():
    return sorted_word_counts

v = get_sorted_word_counts()
print(len(v))

30383


In [136]:
def get_topn_percent(perc):
    counts = get_sorted_word_counts()
    l = len(counts)
    max_index = int(perc*l)
    top_n = []
    #print(counts[:max_index])
    for t in counts[:max_index]:
        top_n.append(t[0])
    return top_n

def get_all_by_max_appearance(app):
    counts = get_sorted_word_counts()
    processed_counts = []
    for c in counts:
        if c[1] >= app:
            processed_counts.append(c[0])
        
    return processed_counts

def get_by_appearance_list(appearance):
    # appearance = [2, 3, 4] # number indicating times of appearance
    counts = get_sorted_word_counts()
    processed_counts = []
    for c in counts:
        if c[1] in appearance:
            processed_counts.append(c[0])
        
    return processed_counts
    
#topn = get_topn_percent(0.001)

selected_appearances = get_by_appearance_list([110, 112])
print(selected_appearances)

#maxapp = get_all_by_max_appearance(2)
#file = open("1106/chat_room_word_appearance.txt", 'w')
#for e in maxapp:
#    try:
#        file.write(e[0] + ", " + str(e[1]) + '\n')
#    except:
#        pass
#file.close()

['girls', 'djkungfukitty']


In [137]:
def remove_non_frequent(allowed_words, post, replace=True):
    good_words = []
    for word in post.lower().split():
        pword = preprocess_word(word)
        if pword in allowed_words:
            good_words.append(pword)
        else:
            if replace:
                replacement = ""
                for char in word:
                    replacement += "-"
                good_words.append(replacement)
    
    return " ".join(good_words)


def remove_by_probability(allowed_words, always_allowed, post, chance, replace=True):
    import random
    good_words = []
    for word in post.lower().split():
        #pword = preprocess_word(word)
        if word in always_allowed:
            good_words.append(word)
            continue
            
        if word in allowed_words:
            good_words.append(word)
        else:
            if replace:
                rv = random.uniform(0.0, 1.0)
                if rv < chance:
                    replacement = ""
                    for char in word:
                        replacement += "-"
                    good_words.append(replacement)
                else:
                    good_words.append(word)
    
    return " ".join(good_words)

sentence_to_test = "Jus to keep being evil<<<< wops Athena on the backside with a new nerf bat"
print(remove_non_frequent(get_topn_percent(0.1), sentence_to_test))

print(remove_by_probability(get_topn_percent(0.1), get_topn_percent(0.01), sentence_to_test, 0.5))

--- to keep being evil ---- athena on the -------- with a new nerf bat
--- to keep being evil<<<< ---- athena on the backside with a new nerf bat


In [214]:
sentence_to_test = "bops saffron on the backside with one of my new nerf bats"

v = 0.0
while v <= 1.0:
    v += 0.04
    print(remove_non_frequent(get_topn_percent(v), sentence_to_test))
    

---- saffron on the -------- with one of my new ---- ----
---- saffron on the -------- with one of my new nerf ----
bops saffron on the -------- with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron on the backside with one of my new nerf bats
bops saffron o

In [5]:
import numpy as np

def get_by_probability(authors, count, topn = 30):
    author_probability = {}
    
    for key, value in authors.items():
        author_probability[key] = len(value)

    author_probability_counts = sorted(author_probability.items(), key=operator.itemgetter(1), reverse=True)
    
    #print(author_probability_counts[:topn])

    sum_sentences = 0
    for v in author_probability_counts[:topn]:
        sum_sentences += v[1]

    author_probability_final = {}
    author_list = []
    prob_list = []

    for v in author_probability_counts[:topn]:
        author_probability_final[v[0]] = v[1]/float(sum_sentences)
        author_list.append(v[0])
        prob_list.append(v[1]/float(sum_sentences))

    final_autor_list = np.random.choice(
      author_list, 
      count,
      p=prob_list
    )

    return final_autor_list

print(get_by_probability(authors, 20))
print(get_by_probability(authors_410, 20))
print(get_by_probability(authors_501, 20))

['sweet_teresa' 'JustMe' 'RoughOne' 'Anastatia' '+DJ`Mercury' 'Malex'
 'Malex' 'Gramps' 'Valkyrie' 'Athena' '@Oscar' 'Rhade' 'FunkyBoogieKing'
 '&Karman' 'slave_of_denial_m' 'FunkyBoogieKing' 'erotic_kitty{S}'
 'RoughOne' 'donna-' 'Anastatia']
['dark-reign' 'Cruel`Intentions' 'Valkyrie' 'carrol' 'Jaems' 'SensualDom'
 'carrol' 'carrol' 'PlayfulBBC' 'carrol' 'Woman' 'Louise' 'carrol'
 'PlayfulBBC' '+DJMercury`' 'Silver_haired`Fox' 'PlayfulBBC' 'saffron{WH}'
 'Valkyrie' 'WhoGiveSaDamn']
['sweet_teresa' '@erotic_kitty' '@DJ`liltech' 'Jothom' '@erotic_kitty'
 '@erotic_kitty{S}' 'sweet_teresa' 'Jothom' '@erotic_kitty'
 '@erotic_kitty{S}' '+DJ`Mercury' 'onetoquiet' '@erotic_kitty{S}'
 'Violatef' '@erotic_kitty{S}' '@erotic_kitty' '@erotic_kitty'
 'FunkyBoogieKing' '+DJ`Mercury' 'FunkyBoogieKing']


In [126]:
# load osc config and parse for later use
import csv

def load_osc_config(file_name):
    config = {}
    with open(file_name, 'r') as csv_file:
        reader = csv.reader(csv_file, delimiter='\t', quotechar='|')
        index = 0
        for row in reader:
            if index > 0:
                #print(row)
                for i in range(len(row)):
                    config[i].append(row[i])
            else:
                config[0] = []
                config[1] = []
                config[2] = []
                config[3] = []
                config[4] = []
            index += 1
    
    return config
            
osc_config = load_osc_config('osc_config.csv')

In [146]:
import random
import sys

def generate_lines_linear(count):
    for i in range(1, count):
        sentence = random.choice(all_posts)
        perc = i/float(count)

        topn = get_topn_percent(perc)
        stripped_post = remove_non_frequent(topn, sentence.strip().lower())
        print(str(perc) + ": " + stripped_post)
        
def generate_lines_from(fromp, step, count):
    for i in range(count):
        sentence = random.choice(all_posts)
        perc = fromp + i * step

        topn = get_topn_percent(perc)
        #print(len(topn))
        stripped_post = remove_non_frequent(topn, sentence[1].strip().lower())
        print("{:.6f}".format(perc) + ": " + stripped_post)
        
def generate_by_osc(config, lists):
    assert(len(config) is not len(lists), "len of config and lists needs to be the same")

    index = 0
    possible_words = []
    # length of generated sentences is determined by osc/curve data
    length = len(config[0])
    for l in lists:
        _config = config[index]
        _min = l[0]
        _max = l[1]
        app_list = []
        for i in range(_min, _max+1):
            app_list.append(i)
        
        # all allowed words
        selected_appearances = get_by_appearance_list(app_list)
        possible_words.append((selected_appearances, _config))
        
        index += 1
        
    always_allowed = get_all_by_max_appearance(500)
    
    test_offset = 90
    
    finished_sentences = []
    for i in range(length):
        #s = random.choice(all_posts)[1]
        s = all_posts[test_offset+i][1]
        for p in possible_words:
            _words = p[0]
            probability = float(p[1][i])
            s = remove_by_probability(_words, always_allowed, s, probability)
        
        finished_sentences.append(s)
    
    return finished_sentences
        

def render_by_osc(out_filename):
    lists = []
    lists.append((1, 5))
    lists.append((6, 20))
    lists.append((21, 50))
    lists.append((51, 100))
    lists.append((101, 500))
    osc_config = load_osc_config('osc_config.csv')
    generated = generate_by_osc(osc_config, lists)

    outfile = codecs.open(out_filename, mode='w', encoding='utf-8')
    for line in generated:
        outfile.write(line + '\n')
    outfile.close()

render_by_osc('chat_curvestest_04.txt')
#generate_lines_linear(20)

#generate_lines_from(0.00, 0.0001, 10)
#print("####################################################################################")
#generate_lines_from(0.10, 0.0001, 10)
#print("####################################################################################")
#generate_lines_from(0.20, 0.0001, 10)
#print("####################################################################################")
#generate_lines_from(0.50, 0.0001, 10)
#print("####################################################################################")
#generate_lines_from(0.80, 0.0001, 10)

<ipython-input-146-a2a3e3d87bde>:24: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(len(config) is not len(lists), "len of config and lists needs to be the same")


In [ ]:
line_count = 100




In [223]:
# postprocess act1&2

for sentence in all_posts_501:
    print(sentence[0] + " -> " + remove_non_frequent(get_topn_percent(0.01), sentence[1]))

Valkyrie -> hello ------------ sir
onetoquiet -> shes ----- ----- ---------------
FunkyBoogieKing -> heya ------------
onetoquiet -> hiyas ------------ ------------- and --------
@erotic_kitty -> heya ------------ ty
Violatef -> hi ------------
jakkiline-cd -> ---------
@erotic_kitty -> oh ------------ why
Violatef -> welcome --------
@erotic_kitty -> wb ------
@erotic_kitty -> radio meltdown djliltech is playing --- ---- ------- - --- for my life
+DJ`Mercury -> --------- ------------ --------- who the -------- and who the --------
[23:04:08] Random_Chaos wondering who the violator and who the violatee -> have a great night
WaywardGentleman -> --------- funkyboogieking --------------- ------------
[23:04:41] FunkyBoogieKing pounce-whomples erotic_kitty -> --------------------------
@erotic_kitty{S} -> while the ----------- ----- is away the ----------- ------ will play
FunkyBoogieKing -> -----
@erotic_kitty{S} -> wb -------
@erotic_kitty{S} -> ty -----
JoyceGG -> --------- ----- ----- 